# STEREO-HAND-TRACKING

### Import

In [11]:
import random
import numpy as np

from src.stereo_matching import Matching
from src.neural_net import NeuralNet

%matplotlib inline

# For auto-reloading external modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data Loading